In [1]:
!pip install mlrose-hiive

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for mlrose-hiive: filename=mlrose_hiive-2.1.3-cp36-none-any.whl size=96601 sha256=e4192e8044bfb2a33469c566cd4aaaf5c1d030439f8e2f1ecb5443347f086247
  Stored in directory: /root/.cache/pip/wheels/ee/ef/8b/941402c2014649279f68a1ed4a37b4d15142dfb9c7e4a6b7ac
Successfully built mlrose-hiive


In [2]:
from mlrose_hiive import NeuralNetwork
from sklearn.metrics import accuracy_score
import csv
import pandas as pd
import time

In [7]:
from mlrose_hiive.algorithms.decay import GeomDecay

def run_neural_network_sa(X_train, y_train, X_test, y_test):
  hidden_nodes = [100, 50, 100]
  acc_pop = []
  acc_pop_t = []

  for decay in [0.99, 0.95, 0.75, 0.5]:
    print("Decay", decay)
    schedule = GeomDecay(init_temp=10, decay=decay)
    start = time.time()
    model = NeuralNetwork(hidden_nodes=hidden_nodes, schedule=schedule, algorithm='simulated_annealing', max_iters=10000, early_stopping=True)
    model.fit(X_train, y_train)
    end = time.time()
    acc_pop_t.append([decay, end-start])
    y_act = model.predict(X_test)
    acc_pop.append([decay, accuracy_score(y_act, y_test)])
    print(acc_pop)

  with open("SA.csv", "w+") as f:
    csv.writer(f).writerows(acc_pop)
  with open("SA_time.csv", "w+") as f:
    csv.writer(f).writerows(acc_pop_t)



In [8]:
from scipy.io.arff import loadarff 
from sklearn.model_selection import train_test_split

raw_data = loadarff('Training Dataset.arff')
ph_data = pd.DataFrame(raw_data[0])

for var in ph_data.columns.values:
    ph_data[var] = pd.Categorical(ph_data[var])
    ph_data[var] = ph_data[var].cat.codes


y = ph_data.Result.values
X = ph_data.drop(['Result'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

run_neural_network_sa(X_train, y_train, X_test, y_test)

Decay 0.99
[[0.99, 0.8593396653098145]]
Decay 0.95
[[0.99, 0.8593396653098145], [0.95, 0.8973315241971959]]
Decay 0.75
[[0.99, 0.8593396653098145], [0.95, 0.8973315241971959], [0.75, 0.882858435097241]]
Decay 0.5
[[0.99, 0.8593396653098145], [0.95, 0.8973315241971959], [0.75, 0.882858435097241], [0.5, 0.8959746720940751]]
